In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import pyrealsense2 as rs
import serial
import numpy as np
import time
from threading import Thread
import threading
import DataDeal265 as dd
import struct
import queue
import cv2
import numpy as np
import ipywidgets
import math
from Fire import ShapeDetector
CopterTakingOff =1
TargetPosition = [0.0,0.0,0.0]

#图像大小
cap_x=360
cap_y=240

# X坐标 Y坐标
# 路径
cp1=[0.0,0.0]#起点
cp2=[-4.1,0.3]#数字识别点一
cp3=[-2.0,2.2]#二维码识别点

cp4=[-2.5,1.7]
cp5=[-2.5,3.5]
cp6=[-2.0,4.0]

cp7=[-1.5,1.7]
cp8=[-1.5,3.5]
cp9=[-2.0,4.0]

cp10=[-2.0,6.0]
cp11=[0.25,6.7]
cp12=[-5.0,6.0]

# 路径点入队
roqu=queue.Queue()
roqu.put(cp1)
roqu.put(cp2)
roqu.put(cp3)
# roqu.put(cp4)
# roqu.put(cp5)
# roqu.put(cp6)

# 图像检测器
sd = ShapeDetector()

font_scale=1.5 #字体大小
font=cv2.FONT_HERSHEY_PLAIN#字体类型
imageFlag=True
CE=0.1#坐标误差判断
KNOWN_DISTANCE = 10 #飞行高度分米

In [ ]:

# 字符串对比
def StrComparison(str1,str2):
    n = len(str1)
    res = []
    for x in str1:
        if x in str2:
            res.append(x)
    #print (n)
    return (n-len(res))

    
#Roi    
def findRoi(frame, thresValue,kernel):
    rois = []
    ret, ddst = cv2.threshold(dst,thresValue,255,cv2.THRESH_BINARY)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.dilate(gray,kernel,iterations=2)
    gray = cv2.erode(gray,kernel,iterations=2)
    # ret, ddst = cv2.threshold(gray1,thresValue,255,cv2.THRESH_BINARY_INV)
    # ret, ddst = cv2.threshold(gray1,thresValue,255,cv2.THRESH_OTSU+cv2.THRESH_BINARY_INV)
    contours, hierarchy = cv2.findContours(gray4,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area = []
    rois = []
    for k in range(len(contours)):
        area.append(cv2.contourArea(contours[k]))
    if area:  
        max_idx = np.argmax(np.array(area))
        x, y, w, h = cv2.boundingRect(contours[max_idx])
        rois.append((x,y,w,h))
    return rois, ddst,area
    

# def CoordinateCorrection(thresValue,kernel):
#     global tarx
#     global tary
#     global routeNodeIndex
#     conter =0
#     while conter<10: 
#         ret, frame=cap.read()    
#         rois, ddst=findRoi(thresValue,kernel)
#         if rois: 
#             x, y, w, h=rois[0]
#             x_small=int(x+(w-h)/2)
#             if  x_small<0:
#                     x_small = 0
#             width=(((x+w/2)-(frame.shape[1]/2)) * KNOWN_DISTANCE)/ 520
#             high=(((y+h/2)-(frame.shape[2]/2)) * KNOWN_DISTANCE)/ 520
#             tarx=tarx+width
#             cv2.rectangle(frame, (x,y), (int(x+w),y+h), (153,153,0), 2)
#             cv2.putText(frame, "%.2fft" % width,
#             (frame.shape[1] - 200, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX,
#             2.0, (0, 255, 0), 3)
#             sum_x+=width/10
#             sum_y+=high/10
#             conter++
#         newEdges = cv2.cvtColor(ddst, cv2.COLOR_GRAY2BGR)
#         newFrame = np.hstack((frame,newEdges))  
#         wid.value = cv2.imencode('.jpg',newFrame)[1].tobytes()
#     tarx=tarx+sum_x/10
#     tary=tary+sum_y/10
#     routeNodeIndex+=1
#     print ("route node %d: x : %.1f , y : %.1f  " %(routeNodeIndex,tarx,tary)

def fireIdentification(cap,wid):
    triangle = 0
    square = 0
    circle = 0
    while True:
        ret,image = cap.read()    
        # 颜色空间转换
        hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)

        #设置阈值
        lower_red = np.array([0, 43, 46])
        upper_red = np.array([10, 255, 255])
        # 根据侷值构建掩模
        mask0=cv2.inRange(hsv,lower_red,upper_red)
        #设置阈值
        lower_red = np.array([156, 43, 46])
        upper_red = np.array([180, 255, 255])
        # 根据侷值构建掩模
        mask1 = cv2.inRange(hsv,lower_red,upper_red)
        # 合并掩模
        mask = mask0 + mask1

        # 稍微模糊，
        blurred = cv2.GaussianBlur(mask, (5, 5), 0)

        # 在阈值图像中查找轮廓并初始化
        # 形状检测器
        cnts,_ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

        # 找到最大轮廓
        max_idx = sd.maxContour(cnts)
        if max_idx: 
            c=cnts[max_idx]
            M = cv2.moments(c)

            # 计算轮廓的中心，然后检测
            #仅使用轮廓的形状
            cX = int((M["m10"] / M["m00"]))
            cY = int((M["m01"] / M["m00"]))
            shape = sd.detect(c)
            
            # 多次判断结果
            if shape="triangle":
                triangle++
                if triangle>10:
                    print(shape)
                    break
            if shape="square":
                square++
                if square>10:
                    print(shape)
                    break
            if shape="circle":
                circle++
                if circle>10:
                    print(shape)
                    break

            # 在图像上绘制轮廓和形状名称
            cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
            cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, (255, 255, 255), 2)

        # 显示输出图像
        sd.showImage(mask,image,wid)


#路径点更新图像识别线程
def Router(cap,wid):
    global routeNodeIndex
    global SendTargetPos
    global CopterLanding
    global LaserArray
    global LaserDistance
    global FlightMode
    global CopterTakingOff
    global routeStartFlag
    global targetx  
    global targety
    while True:
        SendTargetPos = 1       
        if math.sqrt((targetx - pos_Y)**2+(targety + pos_X)**2)<CE and routeStartFlag:
            if imageFlag:
                if math.sqrt((tartargetx - pos_Y)**2+(targety + pos_X)**2)<CE:
                    fireIdentification(cap,wid)
                   
            if not roqu.empty():
                TargetPosition=roqu.get()
                routeNodeIndex+=1
                targetx = float(TargetPosition[0])
                targety = float(TargetPosition[1])
                print ("route node %d: x : %.1f , y : %.1f  " %(routeNodeIndex, targetx, targety))
                time.sleep(1)
            else:
                # pipe.stop()
                #2SendTargetPos = 0
                CopterTakingOff = 1
                CopterLanding = 1
                routeNodeIndex= 1
                print ("Landing") 
                pipe.stop()
                time.sleep(1.0)  
                break
    

           

#串口通信线程
def PortCom(port):
    global pipe
    global cfg
    global SendTargetPos
    global CopterLanding
    global CopterTakingOff
    global _265Ready
    global GetOnceCmd
    global routeNodeIndex
    global routeStartFlag

    while(True):
#         size=port.inWaiting()
#         if(size!=0):

        response = port.readline()
        if(response !=None):
            port.flushInput()        
            CmdStr1 = str(b'Start265\n')
            CmdStr2 = str(b'Departures\n')
            CmdStr3 = str(b'Refresh265\n')
            CMD = str(response)            
            #刷新265
            if( ( StrComparison( CMD ,CmdStr1 )<=1)  and  GetOnceCmd ==False):
                print(StrComparison( CMD ,CmdStr1 ),response,CMD)
                # Declare RealSense pipeline, encapsulating the actual device and sensors
                pipe = rs.pipeline()
#                 try:
#                     pipe.stop()
#                 except:
#                     print("Error1")
                # Build config object and request pose data
                cfg = rs.config()
                cfg.enable_stream(rs.stream.pose,rs.format.any,framerate=200)
                # 使用请求的配置开始流式传输
                pipe.start(cfg)
                dd.initData()
                SendTargetPos = 0
                CopterLanding = 0
                _265Ready=True
                GetOnceCmd =True 
                routeStartFlag = True

            elif( ( StrComparison(CMD ,CmdStr2 )<=1)  and  CopterTakingOff == 1 ):
                print(StrComparison(CMD ,CmdStr2 ),response,CMD)
                print("Get!")
                # 打开摄像头  
                cap,wid = sd.openCamera()
                router = Thread(target=Router, args=(cap,wid,))
                router.start()
                CopterTakingOff = 0
                
            elif( StrComparison(CMD ,CmdStr3 )<=1):
                _265Ready = False
                GetOnceCmd = False
                routeNodeIndex= 1
                CopterTakingOff =1
                routeStartFlag = False
                print("ReStart!")
                print( StrComparison(CMD ,CmdStr3 ),response,CMD)
                try:
                    pipe.stop()
                    time.sleep(1.0)               
                except:
                    print("Error2")  
            response = 0
            CMD = 0
       
                        
        time.sleep(0.02) 

if __name__ == '__main__':
    global routeNodeIndex # 路由节点索引
    global SendTargetPos  #发送目标位置
    global CopterLanding  # 降落 
    global LaserArray     # 雷达
    global LaserDistance  # 激光器距离
    global FlightMode     #飞行模式
    global pipe
    global _265Ready      # 265准备就绪
    global GetOnceCmd     # 获取一次命令
    global CheckSum       # 校验和
    global targetx        # 目标x
    global targety        # 目标y
    global dataBuf

    # 串口
    port = serial.Serial(port="/dev/ttyAMA0",baudrate=230400,stopbits=1,parity=serial.PARITY_NONE,timeout=1000)
    kernel = np.ones((4, 4), np.uint8) #膨胀算子
    thresValue=50                      #阈值
    


    #串口通信线程
    thread_Serial = Thread(target=PortCom, args=(port,))
    thread_Serial.start()
    
    #导入
    TargetPosition=roqu.get()
    targetx = TargetPosition[0]
    targety = TargetPosition[1]
    
    routeNodeIndex = 1
    _265Ready =False
    GetOnceCmd =False
    CheckSum =0 
    dataBuf = [0]*65
    LaserArray =0
    LaserDistance=0.0
    FlightMode =0
    try:
        while(True):
            if _265Ready:
                # 等待t265相机的下一组帧
                frames = pipe.wait_for_frames()
                # 获取姿势帧
                pose = frames.get_pose_frame()
                if pose:   
                    # 将一些姿势数据打印到终端
                    data = pose.get_pose_data()
                    dataBuf,pos_X,pos_Y,pos_Z,Euler=dd.solveData(data)
                    if(SendTargetPos == 1):           
                        posZ=0.0
                        dataBuf[43] = 0x20
                        posX_buf = struct.pack("f",targetx) 
                        dataBuf[44] = posX_buf[0]
                        dataBuf[45] = posX_buf[1]
                        dataBuf[46] = posX_buf[2]
                        dataBuf[47] = posX_buf[3] 
                        posY_buf = struct.pack("f",targety)
                        dataBuf[48] = posY_buf[0]
                        dataBuf[49] = posY_buf[1]
                        dataBuf[50] = posY_buf[2]
                        dataBuf[51] = posY_buf[3]
                        posZ_buf = struct.pack("f",posZ) 
                        dataBuf[52] = posZ_buf[0]
                        dataBuf[53] = posZ_buf[1]
                        dataBuf[54] = posZ_buf[2]
                        dataBuf[55] = posZ_buf[3]

                        dataBuf[56] = LaserArray              
                        Laser_Dis = struct.pack("f",LaserDistance)
                        dataBuf[57] = Laser_Dis[0]
                        dataBuf[58] = Laser_Dis[1]
                        dataBuf[59] = Laser_Dis[2]             
                        dataBuf[60] = Laser_Dis[3]                  
                        dataBuf[61] = FlightMode
                   
                    if CopterLanding==1:
                        dataBuf[62] = 0xA5
                    else:
                        dataBuf[62] = 0x00
                    
                    for i in range(0,62):                     
                        CheckSum =  CheckSum + dataBuf[i]
                
                    dataBuf[63] = 0xAA
                    dataBuf[64] = CheckSum & 0x00ff
                    
                    print("\rrpy_rad[0]:{:.2f},rpy_rad[1]:{:.2f},rpy_rad[2]:{:.2f} ,X:{:.2f},Y:{:.2f},Z:{:.2f} ".format(Euler[0]*57.3,Euler[1]*57.3,Euler[2]*57.3,pos_Y,-pos_X,pos_Z),end="")
                    port.write(dataBuf)
                    CheckSum = 0
#                     pipe.stop()
            else:
                dataBuf[0] = 0x55
                dataBuf[1] = 0xAA
                dataBuf[2] = 0xFF
                dataBuf[63] = 0xAA
                dataBuf[64] = 0x00
                port.write(dataBuf)
                time.sleep(0.1)
            # wid.value = cv2.imencode('.jpg',Frame)[1].tobytes()
    finally:
        
        pipe.stop()
        dataBuf[62] = 0xA5
        port.write(dataBuf)
        print("some erro")


Image(value=b'')